1. await может быть использован не только при вызове корутина, но и любого эуэйтбл объекта

2. эуэйтбл объект - это объект реализующий дандер метод __awaitable__(), который возвращает итератор

3. корутин относится к эуэйтбл обектам

4. к эуэитбл объектам также относятся объета Task и Future. Таски оборачивают ф-и корутины, типа более высокоуровневая история, для них также необходим синхронный цикл обработки событий, как и для корутинов, а Фьюче - специальный низкоуровневый тип, типа загрушка, который представляет собой будущий результат выполнения асинхоронной ф-м (корутина). Некоторый библиотеки возвращают из ф-й такие объекы, такие ф-и мы может эуэйтить.

Вообще можно сказать, что таски и корутины - это прям максимально родственные темы



Таски или ф-и корутины, которые были поставлены в очередь на исполнение, но еще не запущены, находятся в статусе Pending, когда они запущены - их состояние Running, а после завершения Фьючер переходит в состояние Done
Не обязательно, что после завершения мы получим какой-то нормальный результат, 
Если в процессе выполнения таска или корутина возникло ислючение, то оно будет проброшено на этапе взятия результата, а если исключения не было, то мы заберем результат, если обратимся к фьючеру

Таски, в отличие от тредов, можно отменять, если мы отменили таск, то ассоциированный с ней фьючер переходит в состояние Cancelled, если из Cancelled Future мы попробуем забрать результат то получим CancelledError, 

а если мы попробуем забрать результат в статусе пендинг или раннинг, то получим стейтмент еррор
___
одним словом, результат надо забирать вовремя и из состояние Дан. На статусы мы в общем-то можем смотреть


Future!!!
___
asyncio.Future это не то же самое, что и concurrent.futures.Future, а первым можно использовать эуэйт, а со вторым нельзя, у них частично или полностью различаются методы. Т.е. у второго есть as_completed(), а у первого нет
___
в общем и целом мы редко сами создаем фьюче и редко манипулируем его состоянием, обычно мы сами создаем и манипулируем в низкоуровневом коде, который используем на колбэках, но результат типа не забираем, 
___
у фьюче есть свой АПИ, фьюче - это некий бокс, временная заглушка, в которую мы кладем результат:

    1. результат этот мы туда кладем при помощи ф-и set_result()
    2. set_exception(exception) - положить исключение
    3. done() / cancelled() - изменение состояния, дан возвращает тру, если фьючер был отменен или на нем вызвали set_result() или set_exception(), а кенселд вернет тру, если был отменен
    4. после завершения фьючера мы можем вызвать ф-ю какую-то через метод add_done_callback(callback)
    5. result() возвращает результат, если его записали через сет_резалт, сет_эксепшн выбросит исключение в этом случае, если мы вызываем рещультат на кенселд, то выскочит кэнслд_ерор, а если фьючер еще не завершен, то получим инвалид_стэйт_ерор
    6. exception() если мы хотим взять на исключение, которые было указано во фьючере, но только если этот эксепшн был установлен, а сам фьючер находится в состоянии дан, ошибки те же как и в пятом
    7. get_loop() - позволяет взять луп, к которому привязан данный фьючер, это тоже метод уровня инстанции


Task
таск наследует фьючер и является оберткой для него, таск позволяет запланировать(запустить) корутину, потом дождаться ее завершения и забрать результат там, где это удобно, можем поветить колбэк на завершение, проверять его состояние и тд, таски удобнее,и с ними мы будем работать чаще, чем с фьючерами

    loop.create_task(fn()) для того, чтобы создать таск, надо вызвать метод 

    loop.ensure_future(fn()) - то же самое, как и креэйт таск, но может еще и принимать Future
    
    done, pending = await asyncio.wait(iterable) - запускает корутины. Можно вызывать метод уэйт, который принимает список корутинов, эти корутины заворачиваются в таски, и применяя в нему await мы получаем два списка - задачи, которые завершились, и те, которые находятся в очереди на исполнение, возвращает беспорядочно, надо вручную контролить, какие результаты принадлежат каким таскам
    
    results await asyncio.gather(iterable) - возвращает список результатов из запущенных корутин, возвращает их в порядке следования корутин в аргументе iterable,
    
    for t in asyncio.as_completed((t1, t2)):
    result = await t  - тут мы получаем результаты из запущенных корутин как только они приходят, опять же неупрядоченные

asyncio & thread-safety


thread-safety значит, что код потокобезопасен, если его без каких-либо ошибок использует несколько потоков одновременно, 

большинство asyncio объектов, такие как фьючер и таски, потоконебезопасны. Зачастую мы не можем увидеть попорченное состояние фьчерса, если мы из разных потоков с ним работаем, если вызовы синхронизированны и расставлены неправильно, то будет в части случаев вылезать ошибка ран тайм ерор, и речь тут не о типа threading.Lock(), а о том, чтобы методы, которые работают с фьючер, вызывались из того ивент лупа, которым фьючер и был создан

asyncio не любит, когда разные потоки манипулируют одним и тем же фьючером.
То, что они непотокобезопасны - по сути - это не проблема, но если речь идет о манипуляции состоянием таких объектов, то потребуется маршалинг вызова. то есть чтобы какой поток создал, тот и манипулировал, а если левому захочется изменить состояние, то этот левый поток отправляет запрос или как это назвать в другой поток, который уже сам изменит состояние

ивентлуп работает в мэйн треде и запускает все колбэки и таски на этом же треде. Этот ивентлуп сам занимается тамслайсингом. В каждом ивентлупе в мэйнтреде исполняется один таск.

когда в первой корутине достигается await другой корутины или таск или хз еще что, первая корутина ставится на паузу,

чтобы поставить в план колбэк, т.н. очередь на исполнение,  из другого потока, надо использовать метод loop.call_soon_threadsafe(callback, *args)

а чтобы поставить корутину в план - loop.run_coroutine_threadsafe(coro())